In [50]:
import numpy as np
import pandas as pd
import xarray as xr
import scipy as sp
import scipy.optimize as opt
import scipy.stats as st
import matplotlib.pyplot as plt
import multiprocessing as mp
import numba as nb
import rasterio
import rioxarray as rxr
#%config InlineBackend.figure_format = 'svg'
import matplotlib as mpl
mpl.rcParams['font.family'] = ['SimSun']

In [41]:
def read_precip():
    filename_lst = []
    base_dir = "../../outputs/precip-cutted/"
    for i in range(0,6):
        year = 2000 + i*3
        for j in range(1,37):
            month = j
            filename_lst.append(base_dir + "_" + str(year) + "_" +str(year+2)+"_"+ str(month) + ".tif")
    filename_lst = filename_lst[0:192]
    
    data_frame = []
    for i in filename_lst:
        print(i)
        test_precip_frame = xr.open_dataset(i).squeeze("band",drop=True)
        test_precip_frame_resampled = test_precip_frame.rio.reproject(test_precip_frame.rio.crs,shape=(96,84),resampling=rasterio.enums.Resampling.bilinear)
        data_frame.append((test_precip_frame_resampled.band_data*0.1).to_numpy())
    return np.array(data_frame)

def read_pet():
    filename_lst = []
    base_dir = "../../outputs/pet-resampled/"
    for i in range(2000,2016):
        year = i
        for j in range(1,13):
            month = j
            filename_lst.append(base_dir + str(year) + "_" + str(month).zfill(2) + ".tif")

    data_frame = []
    for i in filename_lst:
        print(i)
        test_pet_frame = xr.open_dataset(i).squeeze("band",drop=True)
        test_pet_frame_resampled = test_pet_frame.rio.reproject(test_pet_frame.rio.crs,shape=(96,84),resampling=rasterio.enums.Resampling.bilinear)
        data_frame.append(test_pet_frame_resampled.band_data.to_numpy())
    return np.array(data_frame)


In [42]:
precip_monthly_mat=read_precip()
pet_monthly_mat=read_pet()

../../outputs/precip-cutted/_2000_2002_1.tif
../../outputs/precip-cutted/_2000_2002_2.tif
../../outputs/precip-cutted/_2000_2002_3.tif
../../outputs/precip-cutted/_2000_2002_4.tif
../../outputs/precip-cutted/_2000_2002_5.tif
../../outputs/precip-cutted/_2000_2002_6.tif
../../outputs/precip-cutted/_2000_2002_7.tif
../../outputs/precip-cutted/_2000_2002_8.tif
../../outputs/precip-cutted/_2000_2002_9.tif
../../outputs/precip-cutted/_2000_2002_10.tif
../../outputs/precip-cutted/_2000_2002_11.tif
../../outputs/precip-cutted/_2000_2002_12.tif
../../outputs/precip-cutted/_2000_2002_13.tif
../../outputs/precip-cutted/_2000_2002_14.tif
../../outputs/precip-cutted/_2000_2002_15.tif
../../outputs/precip-cutted/_2000_2002_16.tif
../../outputs/precip-cutted/_2000_2002_17.tif
../../outputs/precip-cutted/_2000_2002_18.tif
../../outputs/precip-cutted/_2000_2002_19.tif
../../outputs/precip-cutted/_2000_2002_20.tif
../../outputs/precip-cutted/_2000_2002_21.tif
../../outputs/precip-cutted/_2000_2002_22.t

In [43]:
d_monthly_mat = precip_monthly_mat - pet_monthly_mat
d_monthly_mat.shape

(192, 96, 84)

In [44]:
d_seq_00 = d_monthly_mat[:,0,0]

In [45]:
def fisk(x,kappa,mu,lambda_):
    up = mu*kappa*np.power((lambda_*(x-mu)),(kappa-1))
    down = (1+np.power((lambda_*(x-mu)),kappa))**2
    return up/down

In [54]:
fisk(d_seq_00,0.3,-223,1.6)

/tmp/ipykernel_292920/4022312350.py:2: RuntimeWarning: invalid value encountered in power
  up = mu*kappa*np.power((lambda_*(x-mu)),(kappa-1))
/tmp/ipykernel_292920/4022312350.py:3: RuntimeWarning: invalid value encountered in power
  down = (1+np.power((lambda_*(x-mu)),kappa))**2


array([ -0.04750741,  -0.04810042,  -0.37621033,          nan,
                nan,          nan,          nan,  -1.2435683 ,
                nan,  -0.22799009,  -0.09795648,  -0.0651962 ,
        -0.05063235,  -0.05683468,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,  -0.12048394,  -0.07923722,  -0.04689514,
        -0.06003714,  -0.0607241 ,  -1.2154235 ,  -0.7968747 ,
                nan,          nan,          nan,          nan,
        -0.15502778,  -0.56238693,  -0.10604721,  -0.05097131,
        -0.04492169,  -0.05171092,  -0.23949991,          nan,
                nan,          nan,  -0.3213051 ,          nan,
        -0.33641958,  -0.11199506,  -0.06670376,  -0.05013106,
        -0.05320816,  -0.0549798 ,  -0.5147385 ,          nan,
                nan,          nan,  -0.6280907 ,  -0.25085336,
        -4.1330204 ,  -0.5608361 ,  -0.11197746,  -0.05364003,
        -0.0525863 ,  -0.03794475,          nan,       

In [47]:
L = lambda x:  np.prod(fisk(d_seq_00,x[0],x[1],x[2]))

In [48]:
opt.minimize(L,[1,2,3])

/tmp/ipykernel_292920/4022312350.py:2: RuntimeWarning: invalid value encountered in power
  up = mu*kappa*np.power((lambda_*(x-mu)),(kappa-1))


      fun: 0.0
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([nan,  0.,  0.])
  message: 'NaN result encountered.'
     nfev: 4
      nit: 0
     njev: 1
   status: 3
  success: False
        x: array([1., 2., 3.])

In [51]:
para = st.fisk.fit(d_seq_00)